# Setup
Initializes connections to mongodb.

In [140]:
%load_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup
from pymongo import MongoClient
import pprint
from parser import *
import os
from selenium import webdriver
from time import sleep
import re
from collections import Counter

client = MongoClient("localhost", 27017)
db = client.retraction_watch
retracted = db.retracted

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
d = Counter()

In [36]:
d["S"]

2

In [35]:
d["S"] += 1

### Scrape RetractionWatchDB
Executes scraper to insert into mongodb

In [2]:
driver = webdriver.Chrome("chromedriver")
driver.get("http://retractiondatabase.org")

Select Reason(s) for retraction
 - concerns about data
 - concerns about results
 - conflict of interest
 - doint the right thing
 - euphamisms for misconduct
 - falsification/fabrication of data
 - falsification/fabrication of results
 - misconduct by author

In [29]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
read_to_mongo(soup, retracted)

In [3]:
retracted_pubmed = list(retracted.find({"publication.pubmed_id":{"$exists":1,"$ne":"00000000"}}))

In [4]:
retracted_pubmed[0].keys()

dict_keys(['_id', 'rwdb_id', 'title', 'subjects', 'journal', 'publisher', 'institutions', 'reasons', 'authors', 'publication', 'retraction', 'doc_types', 'notice_type', 'countries', 'notes'])

In [21]:
def trim_url(url):
    url = re.sub(r"^https?://", "", url)
    url = re.sub(r"/.*","",url)
    return url

In [22]:
a = "https://github.com/arstarsin/arosden"
trim_url(a)

'github.com'

In [44]:
stems = Counter()

In [46]:
for i in range(100):
    pubmed_id = retracted_pubmed[i]["publication"]["pubmed_id"]
    sleep(5)
    links = fulltext_links(pubmed_id)
    print(i)
    links = [trim_url(link) for link in links]
    if "www.ncbi.nlm.nih.gov" in links:
        stems["www.ncbi.nlm.nih.gov"] += 1
    else:
        for link in links:
            stems[link] += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [47]:
print(stems)

Counter({'www.ncbi.nlm.nih.gov': 45, 'linkinghub.elsevier.com': 13, 'dx.doi.org': 10, 'Insights.ovid.com': 6, 'jamanetwork.com': 4, 'jcs.biologists.org': 4, 'www.neurology.org': 2, 'www.karger.com?DOI=10.1159': 2, 'diabetes.diabetesjournals.org': 2, 'www.researchprotocols.org': 1, 'archpedi.jamanetwork.com': 1, 'archinte.jamanetwork.com': 1, 'www.reproduction-online.org': 1, 'circheartfailure.ahajournals.org': 1, 'www.tandfonline.com': 1, 'joi.jlc.jst.go.jp': 1, 'journals.sagepub.com': 1, 'www.jbc.org': 1, 'www.liebertonline.com': 1})


In [68]:
pm_ids = [p["publication"]["pubmed_id"] for p in retracted_pubmed]
fulltexts = [get_fulltext(a) for a in pm_ids[:2]]


In [86]:
driver = webdriver.Chrome("chromedriver")

In [88]:
def se_fulltext(pubmed_id):
    links = fulltext_links( pubmed_id )
    for url_stem in known_parsers:
        for url in links:
            if url_stem in url:
                driver.get(url)
                sleep(5)
                html = driver.page_source
                soup = BeautifulSoup( html, "lxml" )
                return known_parsers[url_stem](soup)
    return {"links": links}


In [90]:

def parse_ncbi_fulltext( soup ):
    """
    Soup → Dict
    """
    article_soup = soup.find(class_="jig-ncbiinpagenav")
    return soup

known_parsers = {"ncbi.nlm.nih.gov": parse_ncbi_fulltext}



In [91]:
pm_ids = [p["publication"]["pubmed_id"] for p in retracted_pubmed]
fulltexts = [se_fulltext(a) for a in pm_ids[:10]]


In [98]:
articles = [a for a in fulltexts if type(a) != dict]

In [101]:
[type(a) for a in articles]

[bs4.element.Tag,
 bs4.element.Tag,
 bs4.element.Tag,
 bs4.element.Tag,
 bs4.element.Tag]

In [72]:
pm_ids[0]

'27687725'

In [100]:
articles = [soup.find(class_="jig-ncbiinpagenav") for soup in articles]

In [104]:
children = [list(a.children) for a in articles]

In [112]:
for i, a in enumerate(children):
    print("\n\nARTICLE {}".format(i))
    for child in a:
            try:
                print (child.attrs)
            except AttributeError:
                print (child)



ARTICLE 0
{'class': ['fm-sec', 'half_rhythm', 'no_top_margin']}
{'class': ['sec']}
{'id': 'idm139912024031152', 'lang': 'en', 'class': ['tsec', 'sec']}
{'id': '__sec1', 'class': ['tsec', 'sec']}
{'id': '__sec2', 'class': ['tsec', 'sec']}
{'id': '__sec10', 'class': ['tsec', 'sec']}
{'id': '__sec11', 'class': ['tsec', 'sec']}
{'id': '__sec25', 'class': ['tsec', 'sec']}
{'id': 'idm139912012260944', 'class': ['tsec', 'sec', 'headless', 'whole_rhythm']}
{'id': 'idm139912028433408', 'class': ['tsec', 'sec', 'headless', 'whole_rhythm']}
{'id': 'idm139912096824288', 'class': ['tsec', 'sec']}
{'style': 'display: none; width: 200px; top: -100px; left: -100px;', 'aria-live': 'assertive', 'aria-hidden': 'true', 'class': ['ui-helper-reset', 'ui-ncbipopper-wrapper', 'ui-ncbilinksmenu']}


ARTICLE 1
{'class': ['fm-sec', 'half_rhythm', 'no_top_margin']}
{'class': ['sec']}
{'id': 'idm140263020463216', 'lang': 'en', 'class': ['tsec', 'sec']}
{'id': '__sec1', 'class': ['tsec', 'sec']}
{'id': '__sec2', 

In [133]:
sections = [s for s in fulltexts[0].find_all(class_="tsec")]

In [134]:
sections[0]

<div class="tsec sec" id="idm139912024031152" lang="en"><div class="goto jig-ncbiinpagenav-goto-container"><span role="menubar"><a aria-expanded="false" aria-haspopup="true" class="tgt_dark page-toc-label jig-ncbiinpagenav-goto-heading" href="#" role="menuitem" title="Go to other sections in this page">Go to:</a></span></div><h2 class="head no_bottom_margin ui-helper-clearfix" id="idm139912024031152title">Abstract</h2><!--article-meta--><div><p class="p p-first-last" id="__p2">Peroxisome proliferator-activated receptor γ (PPARγ) and runt-related transcription factor 2 (RUNX2) are key regulators of mesenchymal stem cell (MSC) differentiation toward adipocytes and osteoblasts, respectively. Post-translational modifications of these factors determine their activities. Dephosphorylation of PPARγ at Ser-112 is required for its adipocytic activity, whereas phosphorylation of RUNX2 at serine 319 (Ser-319) promotes its osteoblastic activity. Here we show that protein phosphatase 5 (PP5) recipr

In [138]:
print(5)

5


In [145]:
parse_ncbi_abstract( sections[0])

adipocyte, bone, chaperone, mesenchymal stem cells (MSCs), osteoblast, osteocyte, peroxisome proliferator-activated receptor (PPAR), protein phosphatase


{'abstract': 'Peroxisome proliferator-activated receptor γ (PPARγ) and runt-related transcription factor 2 (RUNX2) are key regulators of mesenchymal stem cell (MSC) differentiation toward adipocytes and osteoblasts, respectively. Post-translational modifications of these factors determine their activities. Dephosphorylation of PPARγ at Ser-112 is required for its adipocytic activity, whereas phosphorylation of RUNX2 at serine 319 (Ser-319) promotes its osteoblastic activity. Here we show that protein phosphatase 5 (PP5) reciprocally regulates each receptor by targeting each serine. Mice deficient in PP5 phosphatase have increased osteoblast numbers and high bone formation, which results in high bone mass in the appendicular and axial skeleton. This is associated with a substantial decrease in lipid-containing marrow adipocytes. Indeed, in the absence of PP5 the MSC lineage allocation is skewed toward osteoblasts and away from lipid accumulating adipocytes, although an increase in beige

In [148]:
[[p.text for p in sec.find_all("p")] for sec in sections]

[['Peroxisome proliferator-activated receptor γ (PPARγ) and runt-related transcription factor 2 (RUNX2) are key regulators of mesenchymal stem cell (MSC) differentiation toward adipocytes and osteoblasts, respectively. Post-translational modifications of these factors determine their activities. Dephosphorylation of PPARγ at Ser-112 is required for its adipocytic activity, whereas phosphorylation of RUNX2 at serine 319 (Ser-319) promotes its osteoblastic activity. Here we show that protein phosphatase 5 (PP5) reciprocally regulates each receptor by targeting each serine. Mice deficient in PP5 phosphatase have increased osteoblast numbers and high bone formation, which results in high bone mass in the appendicular and axial skeleton. This is associated with a substantial decrease in lipid-containing marrow adipocytes. Indeed, in the absence of PP5 the MSC lineage allocation is skewed toward osteoblasts and away from lipid accumulating adipocytes, although an increase in beige adipocyte 

In [111]:
for i, a in enumerate(children):
    print("\n\nARTICLE {}".format(i))
    for b in a:
        for child in b:
            try:
                print (child.attrs)
            except AttributeError:
                print (child)



ARTICLE 0
{'class': ['fm-citation', 'half_rhythm', 'no_top_margin', 'clearfix']}
{'class': ['content-title']}
{'class': ['half_rhythm']}
{'class': ['fm-panel', 'small', 'half_rhythm']}
{'id': 'pmclinksbox', 'class': ['links-box', 'whole_rhythm']}
{'class': ['goto', 'jig-ncbiinpagenav-goto-container']}
{'class': ['head', 'no_bottom_margin', 'ui-helper-clearfix'], 'id': 'idm139912024031152title'}
article-meta
{}
{'class': ['sec']}
{'class': ['goto', 'jig-ncbiinpagenav-goto-container']}
{'class': ['head', 'no_bottom_margin', 'ui-helper-clearfix'], 'id': '__sec1title'}
{'id': '__p3', 'class': ['p', 'p-first']}
{'id': '__p4'}
{'id': '__p5', 'class': ['p', 'p-last']}
{'class': ['goto', 'jig-ncbiinpagenav-goto-container']}
{'class': ['head', 'no_bottom_margin', 'ui-helper-clearfix'], 'id': '__sec2title'}
{'id': '__sec3', 'class': ['sec', 'sec-first-last']}
{'class': ['goto', 'jig-ncbiinpagenav-goto-container']}
{'class': ['head', 'no_bottom_margin', 'ui-helper-clearfix'], 'id': '__sec10titl

In [153]:
get_fulltext(pubmed_id)

{'abstract': 'The relationship of elevated depression risk recently discovered among adult persons raised by same-sex parents with possible precipitating conditions in childhood has not previously been acknowledged. This study tests whether such inattention is supportable. Logistic regression based risk ratios were estimated from longitudinal measures of mental health outcomes observed in three waves (at ages 15, 22, and 28) of the US National Survey of Adolescent to Adult Health (n = 15,701). At age 28, the adults raised by same-sex parents were at over twice the risk of depression (CES-D: risk ratio 2.6, 95% CI 1.4–4.6) as persons raised by man-woman parents. These findings should be interpreted with caution. Elevated risk was associated with imbalanced parental closeness and parental child abuse in family of origin; depression, suicidality, and anxiety at age 15; and stigma and obesity. More research and policy attention to potentially problematic conditions for children with same-s

In [85]:
fulltexts[0]

<?xml version="1.0" encoding="utf-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- AppResources meta begin -->
<script type="text/javascript">var ncbi_startTime = new Date();</script>
<!-- AppResources meta end -->
<!-- TemplateResources meta begin -->
<meta content="error" name="paf_template"/>
<!-- TemplateResources meta end -->
<!-- Logger begin -->
<meta content="pmc" name="ncbi_app"/><meta content="error" name="ncbi_pdid"/>
<!-- Logger end -->
<title>Page not available</title>
<!-- AppResources external_resources begin -->
<link href="/core/jig/1.14.8/css/jig.min.css" rel="stylesheet"/><script src="/core/jig/1.14.8/js/jig.min.js" type="text/javascript"></script>
<!-- AppResources external_resources end -->
<!-- Page meta begin -->
<meta content="NOINDEX,NOFOLL

In [70]:
[n.attrs for n in fulltexts[1].find_all(class_="sec")]

[{'class': ['sec']},
 {'class': ['tsec', 'sec'], 'id': 'idm139627154362192', 'lang': 'en'},
 {'class': ['sec']},
 {'class': ['tsec', 'sec'], 'id': '__sec1'},
 {'class': ['tsec', 'sec'], 'id': '__sec2'},
 {'class': ['sec', 'sec-first'], 'id': '__sec3'},
 {'class': ['sec'], 'id': '__sec4'},
 {'class': ['sec'], 'id': '__sec5'},
 {'class': ['sec'], 'id': '__sec6'},
 {'class': ['sec'], 'id': '__sec7'},
 {'class': ['sec'], 'id': '__sec8'},
 {'class': ['sec', 'sec-last'], 'id': '__sec9'},
 {'class': ['tsec', 'sec'], 'id': '__sec10'},
 {'class': ['sec', 'sec-first'], 'id': '__sec11'},
 {'class': ['sec'], 'id': '__sec12'},
 {'class': ['sec'], 'id': '__sec13'},
 {'class': ['sec'], 'id': '__sec14'},
 {'class': ['sec', 'sec-last'], 'id': '__sec15'},
 {'class': ['tsec', 'sec'], 'id': '__sec16'},
 {'class': ['tsec', 'sec'], 'id': 'idm139627152227984'},
 {'class': ['sec']},
 {'class': ['tsec', 'sec'], 'id': 'idm139627157974048'},
 {'class': ['ref-list-sec', 'sec'], 'id': 'reference-list'}]

In [67]:
[ n.attrs for n in list(fulltexts[1].children)]

[{'class': ['fm-sec', 'half_rhythm', 'no_top_margin']},
 {'class': ['sec']},
 {'class': ['tsec', 'sec'], 'id': 'idm139953541649152', 'lang': 'en'},
 {'class': ['tsec', 'sec'], 'id': '__sec1'},
 {'class': ['tsec', 'sec'], 'id': '__sec2'},
 {'class': ['tsec', 'sec'], 'id': '__sec10'},
 {'class': ['tsec', 'sec'], 'id': '__sec16'},
 {'class': ['tsec', 'sec'], 'id': 'idm139953548534096'},
 {'class': ['tsec', 'sec'], 'id': 'idm139953548533088'}]

In [43]:
for i in stems:
    print(i, stems[i])

www.jbc.org 3
www.ncbi.nlm.nih.gov 24
www.scielo.br 1
linkinghub.elsevier.com 10
www.researchprotocols.org 1
Insights.ovid.com 2
publichealth.jmir.org 1
dx.doi.org 9
dx.plos.org 5
academic.oup.com 1
jamanetwork.com 4
archpedi.jamanetwork.com 1
archinte.jamanetwork.com 1
ejbc.kr 1
www.neurology.org 2
mcb.asm.org 4


In [25]:
retracted_pubmed[36]["publication"]["pubmed_id"]

'9371907'

In [7]:
pubmed_sample = fulltext_links( retracted_pubmed[0]["publication"]["pubmed_id"] )

['http://www.jbc.org/cgi/pmidlookup?view=long&pmid=27687725',
 'https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/27687725/']

In [71]:
anchors = (pubmed_sample.find("div", class_="icons portlet").find_all("a"))

In [79]:
[a.attrs["href"] for a in anchors]

['http://www.jbc.org/cgi/pmidlookup?view=long&pmid=27687725',
 'https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/27687725/']

In [76]:
retracted

<a free_status="free" href="http://www.jbc.org/cgi/pmidlookup?view=long&amp;pmid=27687725" journal="J Biol Chem" ref="PrId=3051&amp;itool=Abstract-def&amp;uid=27687725&amp;nlmid=2985121R&amp;db=pubmed&amp;log$=linkouticon" target="_blank"><img alt="Icon for HighWire" border="0" src="//www.ncbi.nlm.nih.gov/corehtml/query/egifs/http:--highwire.stanford.edu-icons-externalservices-pubmed-standard-jbc_final_free.gif" title="Read full text in HighWire"/></a>

In [78]:
a.attrs["href"]

'http://www.jbc.org/cgi/pmidlookup?view=long&pmid=27687725'

In [4]:
rows = soup.find_all(attrs={"class":"mainrow"})

In [6]:
print(len(rows))

277


In [6]:
print (rows[9].prettify ())

<tr class="mainrow" style="background-color:White;">
 <td class="hidden smallFont" style="color:Black;">
  4376
 </td>
 <td class="smallFont" id="grdRetraction_Title_9" style="border-color:Silver;" title="4376">
  <a href="javascript:__doPostBack('grdRetraction','Select$4376')" onclick="return false;" style="cursor:default;">
   <span class="rTitleNotIE">
   </span>
   <br/>
   <span class="rSubject">
    (BLS) Biology - Cancer; (HSC) Medicine - Oncology; (HSC) Medicine - Pathology; (HSC) Medicine - Urology;
   </span>
   <br/>
   <span class="rJournal" style="white-space:normal">
    <span class="rJournal">
     Diagnostic Pathology ---
    </span>
    <span class="rPublisher">
     BioMed Central (BMC)
    </span>
   </span>
   <br/>
   <span class="rInstitution" style="border-bottom: 2px solid rgb(0, 102, 33); border-top-style: solid; border-top-width: 2px;">
    Department of Urology, Henan Provincial People’s Hospital, Zhengzhou
   </span>
   <br/>
  </a>
  <a class="urls" href="h

In [10]:
rows[0].find(class_="rPublisher").text

'Spandidos'

In [8]:
a = [5,3,41]

In [13]:
print(a.pop())
print(a)

IndexError: pop from empty list

In [5]:
a = rows[30].find(class_="authorLink").parent
p = a.find_next_sibling()
r = p.find_next_sibling()
t = r.find_next_sibling()
f = t.find_next_sibling()
list(f.children).pop(-2)

<img alt="NotesAdmin" border="0" class="NotesAdmin" id="imgNotesAdmin" src="../Images/NotesAdmin.png" style="display:none;margin-top:2px;" title=""/>

In [16]:
acc = ""
for b in list(p.children):
    acc = acc + str(b)

In [31]:
[type(ch) for ch in list(p.children)]
[ch for ch in list(p.children) if ch.name != "br"]

['08/20/2009',
 '19693644',
 <span class="rNature">10.1007/s11845-009-0394-5</span>]

In [24]:
[(a) for a in list(p.children) if str(a) == a]


['07/25/2016']

In [12]:
read_table(soup)

AttributeError: 'NavigableString' object has no attribute 'text'

In [129]:
rows[0].attrs

{'class': ['mainrow'],
 'style': 'background-color:LightYellow;border-color:Black;border-width:1px;border-style:Solid;'}

In [37]:
rows[0].find(id=lambda x: x and x.startswith("grdRetraction_Title")).attrs

{'class': ['smallFont'],
 'id': 'grdRetraction_Title_0',
 'style': 'border-color:Silver;',
 'title': '18030'}

In [92]:
a = rows[1].find(class_="authorLink").parent

In [93]:
p = a.find_next_sibling()
r = p.find_next_sibling()
t = r.find_next_sibling()
f = t.find_next_sibling()

In [94]:
rows[0].find(alt="NotesAdmin").attrs["title"]

'vicky handling rolf  https://academic.oup.com/carcin/advance-article/doi/10.1093/carcin/bgy030/4955856\n\nCarcinogenesis is retracting the following paper following a joint research misconduct investigation by the Birmingham VA Medical Center and the University of Alabama at Birmingham, AL, USA.\n\nNandakumar, V. et al. (2011) Aberrant DNA hypermethylation patterns lead to transcriptional silencing of tumor suppressor genes in UVB-exposed skin and UVB-induced skin tumors of mice. Carcinogenesis, 32(4), 597–604.\n\nIn Figure 5, there are several instances in which bands are replicated but labelled to represent different experimental conditions. For example, the bands labeled P16INK4a ‘Ac-Histone H3’ and ‘MBD1’ are the same; the bands labeled P16INK4a ‘Ac-Histone H4 Input’ and ‘MBD1Input’ and ‘HDAC1 Input’ and RASSF1A ‘HDAC1Input’ appear to be identical.\n\nIn some cases, the bands are flipped (compare MeCP2 and HDAC-1 in Panel A). While the Input bands for Panel A are all the same, the

In [95]:
for y in f.children:
    print(y)



<span>Spain<br/>United States<br/><span class="rPaywalled">No</span></span>


<img alt="Notes" border="0" class="Notes" id="imgNotes" src="../Images/Notes.png" style="display:none" title=""/>


<img alt="NotesAdmin" border="0" class="NotesAdmin" id="imgNotesAdmin" src="../Images/NotesAdmin.png" style="display:none;margin-top:2px;" title="staff   https://onlinelibrary.wiley.com/doi/10.1111/clr.13083

Calvo-Guirado JL, Ramírez-Fernández MP, Delgado-Ruíz RA, Maté-Sánchez JE, Velasquez P, de Aza PN. Influence of Biphasic ß-TCP with and without the use of collagen membranes on bone healing of surgically critical size defects. A radiological, histological, and histomorphometric study. Clin Oral Impl Res. 2014;25: 1228–1238.

The above article, published online on 12 September 2013 in Wiley Online Library (wileyonlinelibrary.com), and in Volume 25, pp. 1228-1238, has been retracted by agreement between the journal Editor in Chief, L Heitz-Mayfield, and John Wiley &amp; Sons Ltd. The retract

In [102]:
c = list(f.children)[1]

In [112]:
[list(c.children)[i] for i in range(len(list(c.children))) if i%2 == 0][:-1]

['Spain', 'United States']

In [106]:
list(f.children.children)

AttributeError: 'list_iterator' object has no attribute 'children'

In [118]:
import parser

In [119]:
parser.read_row(rows[0])

AttributeError: 'NoneType' object has no attribute 'text'